# Project Title
### Data Engineering Capstone Project

#### Project Summary
This project contains several files provided by udacity. As the different data could be merged for several reasons i will take out one speciific purpose for data insight generation. As the provided data has some information about us demographics and non-immigrant entrants, it could be interesting to see if there are relationsships to each other. For creating such knowledge it is necessary to build an etl with the neccerssary steps (EDA, cleaning, wrangling, writing,etc.). Afterwards the department can work out the needed information. 

I will use for my setup a star schema as this is the simpliest way ramping up the business and allows fastly generate insights. I will create fact and dimension tables as required in a star schema. 

Steps of the project:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [10]:
# Do all imports and installs here and initiate spark
import pandas as pd
import datetime as dt
import numpy as np

from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
.getOrCreate()

from pyspark.sql.functions import first
from pyspark.sql.functions import upper, col
from pyspark.sql.types import StructField, StructType, StringType, LongType, IntegerType
from pyspark.sql.functions import udf, date_format


### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

I will transform the US Demographic dataset by pivoting on the Race column with the aim to convert Race categories into individual columns. We will achieve this transformation by separating both Race and Count columns from the US demo dataset and initiate two separate datasets called US and US RACE COUNT.

US RACE COUNT dataset will also include City and State Code columns as a basis to join back with US dataset. Before we join the two datasets we will remove duplicates from the US dataset and pivot the US Race Count dataset. In theory both datasets should have equal rows to join them back to single transformed table.

Describe and Gather Data
Describe the data sets you're using. Where did it come from? What type of information is included?

The I94 immigration data comes from the US National Tourism and Trade Office. This data was already provided inside sas_data folder in parquet file type. We will use the following records:
i94Bir - Age of non-immigrant in years, admnum - Admission Number, i94res - 3 digit code of nationality, i94port - 3 character code of destination USA city, arrdate - arrival date in the USA (SAS date numeric field), i94mode - 1 digit mode (plane, boat, etc) of travel code i94visa - reason for immigration, gender - Non-immigrant sex, depdate - departure date in the USA (SAS date numeric field), count - Used for summary statistics

U.S. City Demographic Data comes from OpenSoft in csv file format and will be converted to json. We will use the following records:
City (USA), Male Population, Female Population, Median Age (overall median age within the city population), Total Population, Foreign-Born (number of foreign born residences), State Code (USA state abbreviation of the City column), Race (specifies race category suchas Asian, Alaskan Indian, Black, Hispanic, etc), Count (number of people under specific race category anotated by Race column)

# Rework asked from Mentor
Who is going to use the data, be specific here, you can think of user persona here
For example the airport or the securities to adapt the skill requirements for employees like beeing able to speak another language.
Or extnd homeland securtiy employes if you know that at certain or certain airports, you have more visa applies than anywhere else.

What are the types of question that can be answered using this data model, again be specific
On which Port do we have the most Latino Population?
On which date did we had the most Visa applies?


### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.
pls. see data_information.ipynb

#### Cleaning Steps
Document steps necessary to clean the data
pls. see upcoming steps

##### Read US Cities Demo dataset file

In [11]:
# Read in us-cities-demographics.csv for further processing
us_spark=spark.read.csv("./data/us-cities-demographics.csv", sep=';', header=True)

In [12]:
# Display columns for choice decision
print(us_spark.columns)# Read in us-cities-demographics.csv for further processing

['City', 'State', 'Median Age', 'Male Population', 'Female Population', 'Total Population', 'Number of Veterans', 'Foreign-born', 'Average Household Size', 'State Code', 'Race', 'Count']


In [13]:
us_spark.orderBy("city").show()

+-------+----------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+------+
|   City|     State|Median Age|Male Population|Female Population|Total Population|Number of Veterans|Foreign-born|Average Household Size|State Code|                Race| Count|
+-------+----------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+------+
|Abilene|     Texas|      31.3|          65212|            60664|          125876|              9367|        8129|                  2.64|        TX|American Indian a...|  1813|
|Abilene|     Texas|      31.3|          65212|            60664|          125876|              9367|        8129|                  2.64|        TX|  Hispanic or Latino| 33222|
|Abilene|     Texas|      31.3|          65212|            60664|          125876|              9367|        8129| 

In [14]:
# Check dataset for duplicate rows for the defined aggregation
us_spark.select("City","state","Median Age","male population","female population","total population", \
                  "foreign-born","Average Household Size").orderBy("city").show()

+-------+----------+----------+---------------+-----------------+----------------+------------+----------------------+
|   City|     state|Median Age|male population|female population|total population|foreign-born|Average Household Size|
+-------+----------+----------+---------------+-----------------+----------------+------------+----------------------+
|Abilene|     Texas|      31.3|          65212|            60664|          125876|        8129|                  2.64|
|Abilene|     Texas|      31.3|          65212|            60664|          125876|        8129|                  2.64|
|Abilene|     Texas|      31.3|          65212|            60664|          125876|        8129|                  2.64|
|Abilene|     Texas|      31.3|          65212|            60664|          125876|        8129|                  2.64|
|Abilene|     Texas|      31.3|          65212|            60664|          125876|        8129|                  2.64|
|  Akron|      Ohio|      38.1|          96886| 

In [15]:
# Check subset of `US` dataset that maybe causing dupliate rows
us_spark.select("city","state code","Race","count").orderBy("city").show()

+-------+----------+--------------------+------+
|   city|state code|                Race| count|
+-------+----------+--------------------+------+
|Abilene|        TX|American Indian a...|  1813|
|Abilene|        TX|  Hispanic or Latino| 33222|
|Abilene|        TX|               White| 95487|
|Abilene|        TX|               Asian|  2929|
|Abilene|        TX|Black or African-...| 14449|
|  Akron|        OH|               White|129192|
|  Akron|        OH|  Hispanic or Latino|  3684|
|  Akron|        OH|Black or African-...| 66551|
|  Akron|        OH|               Asian|  9033|
|  Akron|        OH|American Indian a...|  1845|
|Alafaya|        FL|  Hispanic or Latino| 34897|
|Alafaya|        FL|               Asian| 10336|
|Alafaya|        FL|               White| 63666|
|Alafaya|        FL|Black or African-...|  6577|
|Alameda|        CA|               White| 44232|
|Alameda|        CA|American Indian a...|  1329|
|Alameda|        CA|Black or African-...|  7364|
|Alameda|        CA|

##### Transformation of demo dataset file

In [16]:
# Creating 'us_race_count' dataset
us_race_count=(us_spark.select("city","state code","Race","count")
    .groupby(us_spark.City, "state code")
    .pivot("Race")
    .agg(first("Count")))

In [17]:
# Checking dataset
us_race_count.orderBy("city").show()

+------------+----------+---------------------------------+-----+-------------------------+------------------+------+
|        City|state code|American Indian and Alaska Native|Asian|Black or African-American|Hispanic or Latino| White|
+------------+----------+---------------------------------+-----+-------------------------+------------------+------+
|     Abilene|        TX|                             1813| 2929|                    14449|             33222| 95487|
|       Akron|        OH|                             1845| 9033|                    66551|              3684|129192|
|     Alafaya|        FL|                             null|10336|                     6577|             34897| 63666|
|     Alameda|        CA|                             1329|27984|                     7364|              8265| 44232|
|      Albany|        NY|                             1611| 8090|                    31303|              9368| 58368|
|      Albany|        GA|                              4

In [18]:
# Comparing datasets after dropping duplicate rows
print('us_spark', us_spark.count())
print('us_race_count', us_race_count.count())
print('us_race_count drop duplicates', us_race_count.dropDuplicates().count())

us_spark 2891
us_race_count 596
us_race_count drop duplicates 596


In [19]:
#adapt us_spark

In [20]:
us_spark.columns

['City',
 'State',
 'Median Age',
 'Male Population',
 'Female Population',
 'Total Population',
 'Number of Veterans',
 'Foreign-born',
 'Average Household Size',
 'State Code',
 'Race',
 'Count']

In [21]:
uscols=["Number of Veterans","Race","Count"]

In [22]:
# Drop columns we don't need and drop duplicate rows
us=us_spark.drop(*uscols).dropDuplicates()

In [23]:
us.columns

['City',
 'State',
 'Median Age',
 'Male Population',
 'Female Population',
 'Total Population',
 'Foreign-born',
 'Average Household Size',
 'State Code']

In [24]:
# Comparing row count between original and new dataset with dropped duplicate rows
print('us_spark', us_spark.count())
print('us_race_count', us_race_count.count())
print('us', us.count())
# We can see that new cleaned 'us' dataset now matches number of rows with 'us_race_cnt' dataset which we will join together into one 'us' dataset

us_spark 2891
us_race_count 596
us 596


In [25]:
us.show(5)

+---------------+-----------+----------+---------------+-----------------+----------------+------------+----------------------+----------+
|           City|      State|Median Age|Male Population|Female Population|Total Population|Foreign-born|Average Household Size|State Code|
+---------------+-----------+----------+---------------+-----------------+----------------+------------+----------------------+----------+
|Highlands Ranch|   Colorado|      39.6|          49186|            53281|          102467|        8827|                  2.72|        CO|
|           Kent| Washington|      33.4|          61825|            65137|          126962|       38175|                  3.06|        WA|
|        Madison|  Wisconsin|      30.7|         122596|           126360|          248956|       30090|                  2.23|        WI|
|         Denver|   Colorado|      34.1|         341137|           341408|          682545|      113222|                  2.33|        CO|
|         Caguas|Puerto Ric

In [26]:
# now lets join the prepared datasets
us.join(us_race_count, ["city","state code"]).count()
# We now see total rows match both datasets after joining them

596

In [27]:
# Checking data sample
us.join(us_race_count, ["city","state code"]).orderBy("city").show()

+------------+----------+------------+----------+---------------+-----------------+----------------+------------+----------------------+---------------------------------+-----+-------------------------+------------------+------+
|        City|State Code|       State|Median Age|Male Population|Female Population|Total Population|Foreign-born|Average Household Size|American Indian and Alaska Native|Asian|Black or African-American|Hispanic or Latino| White|
+------------+----------+------------+----------+---------------+-----------------+----------------+------------+----------------------+---------------------------------+-----+-------------------------+------------------+------+
|     Abilene|        TX|       Texas|      31.3|          65212|            60664|          125876|        8129|                  2.64|                             1813| 2929|                    14449|             33222| 95487|
|       Akron|        OH|        Ohio|      38.1|          96886|           100667| 

In [28]:
# Finally saving (committing) joined US dataset
us=us.join(us_race_count, ["city","state code"])

In [29]:
# Another check
(us.count(), us.show(5))

+---------------+----------+-----------+----------+---------------+-----------------+----------------+------------+----------------------+---------------------------------+-----+-------------------------+------------------+------+
|           City|State Code|      State|Median Age|Male Population|Female Population|Total Population|Foreign-born|Average Household Size|American Indian and Alaska Native|Asian|Black or African-American|Hispanic or Latino| White|
+---------------+----------+-----------+----------+---------------+-----------------+----------------+------------+----------------------+---------------------------------+-----+-------------------------+------------------+------+
|Highlands Ranch|        CO|   Colorado|      39.6|          49186|            53281|          102467|        8827|                  2.72|                             1480| 5650|                     1779|              8393| 94499|
|           Kent|        WA| Washington|      33.4|          61825|         

(596, None)

In [30]:
# Change `state code` column name to `state_code` and other similar problems to avoid parquet complications
us=us.select('City', col('State Code').alias('State_Code'), 'State', col('Median Age').alias('Median_age'),
     col('Male Population').alias('Male_Pop'), col('Female Population').alias('Fem_Pop'), 
        col('Total Population').alias('Ttl_Pop'), 'Foreign-born', 
          col('Average Household Size').alias('Avg_Household_Size'),
             col('American Indian and Alaska Native').alias('Native_Pop'), 
                 col('Asian').alias('Asian_Pop'), 
                    col('Black or African-American').alias('Black_Pop'), 
                      col('Hispanic or Latino').alias('Latino_Pop'), 
                        col('White').alias('White_Pop'))

In [31]:
# Drop the `state` column
us=us.drop("state")

In [32]:
us.columns

['City',
 'State_Code',
 'Median_age',
 'Male_Pop',
 'Fem_Pop',
 'Ttl_Pop',
 'Foreign-born',
 'Avg_Household_Size',
 'Native_Pop',
 'Asian_Pop',
 'Black_Pop',
 'Latino_Pop',
 'White_Pop']

In [33]:
us.show(2)

+---------------+----------+----------+--------+-------+-------+------------+------------------+----------+---------+---------+----------+---------+
|           City|State_Code|Median_age|Male_Pop|Fem_Pop|Ttl_Pop|Foreign-born|Avg_Household_Size|Native_Pop|Asian_Pop|Black_Pop|Latino_Pop|White_Pop|
+---------------+----------+----------+--------+-------+-------+------------+------------------+----------+---------+---------+----------+---------+
|Highlands Ranch|        CO|      39.6|   49186|  53281| 102467|        8827|              2.72|      1480|     5650|     1779|      8393|    94499|
|           Kent|        WA|      33.4|   61825|  65137| 126962|       38175|              3.06|      3651|    26168|    20450|     21928|    67918|
+---------------+----------+----------+--------+-------+-------+------------+------------------+----------+---------+---------+----------+---------+
only showing top 2 rows



In [34]:
us.printSchema()

root
 |-- City: string (nullable = true)
 |-- State_Code: string (nullable = true)
 |-- Median_age: string (nullable = true)
 |-- Male_Pop: string (nullable = true)
 |-- Fem_Pop: string (nullable = true)
 |-- Ttl_Pop: string (nullable = true)
 |-- Foreign-born: string (nullable = true)
 |-- Avg_Household_Size: string (nullable = true)
 |-- Native_Pop: string (nullable = true)
 |-- Asian_Pop: string (nullable = true)
 |-- Black_Pop: string (nullable = true)
 |-- Latino_Pop: string (nullable = true)
 |-- White_Pop: string (nullable = true)



In [35]:
# Now write (and overwrite) transformed `US` dataset onto parquet file
us.write.mode('overwrite').parquet("./data/us_cities_demographics.parquet")

#### I94 NON-IMMIGRATION DATA SET (CLEANING AND TRANSFORMATION)

In [36]:
# Read i94 non-immigration dataset
i94_spark=spark.read.parquet("./data/sas_data")
i94_spark.columns

['cicid',
 'i94yr',
 'i94mon',
 'i94cit',
 'i94res',
 'i94port',
 'arrdate',
 'i94mode',
 'i94addr',
 'depdate',
 'i94bir',
 'i94visa',
 'count',
 'dtadfile',
 'visapost',
 'occup',
 'entdepa',
 'entdepd',
 'entdepu',
 'matflag',
 'biryear',
 'dtaddto',
 'gender',
 'insnum',
 'airline',
 'admnum',
 'fltno',
 'visatype']

In [37]:
i94_spark.select("i94res","i94port","arrdate","i94mode","depdate","i94bir","i94visa","count" \
                  ,"gender",col("admnum").cast(LongType())).show(3)

+------+-------+-------+-------+-------+------+-------+-----+------+-----------+
|i94res|i94port|arrdate|i94mode|depdate|i94bir|i94visa|count|gender|     admnum|
+------+-------+-------+-------+-------+------+-------+-----+------+-----------+
| 438.0|    LOS|20574.0|    1.0|20582.0|  40.0|    1.0|  1.0|     F|94953870030|
| 438.0|    LOS|20574.0|    1.0|20591.0|  32.0|    1.0|  1.0|     F|94955622830|
| 438.0|    LOS|20574.0|    1.0|20582.0|  29.0|    1.0|  1.0|     M|94956406530|
+------+-------+-------+-------+-------+------+-------+-----+------+-----------+
only showing top 3 rows



In [38]:
i94_spark=i94_spark.select(col("i94res").cast(IntegerType()),col("i94port"),
                           col("arrdate").cast(IntegerType()), \
                           col("i94mode").cast(IntegerType()),col("depdate").cast(IntegerType()),
                           col("i94bir").cast(IntegerType()),col("i94visa").cast(IntegerType()), 
                           col("count").cast(IntegerType()), \
                              "gender",col("admnum").cast(LongType()))

In [39]:
i94_spark.show(3)

+------+-------+-------+-------+-------+------+-------+-----+------+-----------+
|i94res|i94port|arrdate|i94mode|depdate|i94bir|i94visa|count|gender|     admnum|
+------+-------+-------+-------+-------+------+-------+-----+------+-----------+
|   438|    LOS|  20574|      1|  20582|    40|      1|    1|     F|94953870030|
|   438|    LOS|  20574|      1|  20591|    32|      1|    1|     F|94955622830|
|   438|    LOS|  20574|      1|  20582|    29|      1|    1|     M|94956406530|
+------+-------+-------+-------+-------+------+-------+-----+------+-----------+
only showing top 3 rows



In [40]:
print('i94_spark count: ', i94_spark.count())
print('i94_spark duplicates', i94_spark.dropDuplicates().count())


i94_spark count:  3096313
i94_spark duplicates 3096302


In [41]:
i94_spark.dropDuplicates(['admnum']).count()

3075579

In [42]:
# We will drop duplicate rows and save it as final dataset for i94
i94_spark=i94_spark.dropDuplicates()

In [43]:
i94_spark.show(3)

+------+-------+-------+-------+-------+------+-------+-----+------+-----------+
|i94res|i94port|arrdate|i94mode|depdate|i94bir|i94visa|count|gender|     admnum|
+------+-------+-------+-------+-------+------+-------+-----+------+-----------+
|   582|    XXX|  20557|   null|  20558|    34|      2|    1|  null|91904214530|
|   209|    AGA|  20552|      1|   null|  null|      2|    1|     M|47842155333|
|   209|    ATL|  20571|      1|   null|  null|      2|    1|     M|44537883633|
+------+-------+-------+-------+-------+------+-------+-----+------+-----------+
only showing top 3 rows



In [44]:
i94_spark.printSchema()

root
 |-- i94res: integer (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: integer (nullable = true)
 |-- i94mode: integer (nullable = true)
 |-- depdate: integer (nullable = true)
 |-- i94bir: integer (nullable = true)
 |-- i94visa: integer (nullable = true)
 |-- count: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- admnum: long (nullable = true)



#### read in sas _labels for further processing

In [45]:
with open('./data/I94_SAS_Labels_Descriptions.SAS') as f:
    f_content = f.read()
    f_content = f_content.replace('\t', '')
def code_mapper(file, idx):
    f_content2 = f_content[f_content.index(idx):]
    f_content2 = f_content2[:f_content2.index(';')].split('\n')
    f_content2 = [i.replace("'", "") for i in f_content2]
    dic = [i.split('=') for i in f_content2[1:]]
    dic = dict([i[0].strip(), i[1].strip()] for i in dic if len(i) == 2)
    return dic
i94cit_res = code_mapper(f_content, "i94cntyl")
i94port = code_mapper(f_content, "i94prtl")
i94mode = code_mapper(f_content, "i94model")
i94addr = code_mapper(f_content, "i94addrl")
i94visa = {'1':'Business',  '2': 'Pleasure', '3' : 'Student'}

In [46]:
labeldescription = [i94cit_res, i94port, i94mode, i94addr, i94visa]
for i in labeldescription:
    print(dict(list(i.items())[0:5]))

{'582': 'MEXICO Air Sea, and Not Reported (I-94, no land arrivals)', '236': 'AFGHANISTAN', '101': 'ALBANIA', '316': 'ALGERIA', '102': 'ANDORRA'}
{'ALC': 'ALCAN, AK', 'ANC': 'ANCHORAGE, AK', 'BAR': 'BAKER AAF - BAKER ISLAND, AK', 'DAC': 'DALTONS CACHE, AK', 'PIZ': 'DEW STATION PT LAY DEW, AK'}
{'1': 'Air', '2': 'Sea', '3': 'Land', '9': 'Not reported'}
{'AL': 'ALABAMA', 'AK': 'ALASKA', 'AZ': 'ARIZONA', 'AR': 'ARKANSAS', 'CA': 'CALIFORNIA'}
{'1': 'Business', '2': 'Pleasure', '3': 'Student'}


#### Performing cleaning tasks for i94 code lists and creating dimension tables

In [47]:

# Start processing the I9I94_SAS_Labels_Description.SAS to create master i94 code dimensions:

'''
/* I94MODE - There are missing values as well as not reported (9) */
	1 = 'Air'
	2 = 'Sea'
	3 = 'Land'
	9 = 'Not reported' ;
'''
# Create i94mode list
i94mode_data =[[1,'Air'],[2,'Sea'],[3,'Land'],[9,'Not reported']]

# Convert to spark dataframe
i94mode=spark.createDataFrame(i94mode_data)

# Create i94mode parquet file
i94mode.write.mode("overwrite").parquet('./data/i94mode.parquet')

In [48]:
i94mode.printSchema()

root
 |-- _1: long (nullable = true)
 |-- _2: string (nullable = true)



In [49]:
df = pd.DataFrame(list(i94port.items()),columns = ['id','port_city']) 

In [50]:
i94port_df = pd.concat([df, df['port_city'].str.split(', ', expand=True)], axis=1).drop(columns=['port_city', 2]).rename(columns={0: 'city', 1: 'state'})#.set_index('id')

In [51]:
i94port_df.to_csv('./data/i94port.csv')

In [52]:
# Read i94port text file
i94port_df = pd.read_csv('./data/i94port.csv')

In [53]:
i94port_data=i94port_df.values.tolist()

In [54]:
# Now convert list to spark dataframe
# Create a schema for the dataframe
i94port_schema = StructType([
    StructField('id', StringType(), True),
    StructField('port', StringType(), True),
    StructField('port_city', StringType(), True),
    StructField('port_state', StringType(), True)
])
i94port=spark.createDataFrame(i94port_data, i94port_schema)

In [55]:
i94port.printSchema()

root
 |-- id: string (nullable = true)
 |-- port: string (nullable = true)
 |-- port_city: string (nullable = true)
 |-- port_state: string (nullable = true)



In [56]:
i94port.show(10)

+---+----+--------------------+----------+
| id|port|           port_city|port_state|
+---+----+--------------------+----------+
|  0| ALC|               ALCAN|        AK|
|  1| ANC|           ANCHORAGE|        AK|
|  2| BAR|BAKER AAF - BAKER...|        AK|
|  3| DAC|       DALTONS CACHE|        AK|
|  4| PIZ|DEW STATION PT LA...|        AK|
|  5| DTH|        DUTCH HARBOR|        AK|
|  6| EGL|               EAGLE|        AK|
|  7| FRB|           FAIRBANKS|        AK|
|  8| HOM|               HOMER|        AK|
|  9| HYD|               HYDER|        AK|
+---+----+--------------------+----------+
only showing top 10 rows



In [57]:
# CREATE PARQUET FILE
i94port.write.mode('overwrite').parquet('./data/i94port.parquet')

In [58]:
i94res_df = pd.DataFrame(list(i94cit_res.items()),columns = ['id','country']) 
#Remove whitespaces and single quotes
#i94res_df['country']=i94res_df['country'].str.replace("'",'').str.strip()
# Convert pandas dataframe to list (objects which had single quotes removed automatically become string again with single quotes)
i94res_data=i94res_df.values.tolist()

In [59]:
# Now convert list to spark dataframe
# Create a schema for the dataframe
i94res_schema = StructType([
    StructField('id', StringType(), True),
    StructField('country', StringType(), True)
])
i94res=spark.createDataFrame(i94res_data, i94res_schema)

In [60]:
i94res.printSchema()

root
 |-- id: string (nullable = true)
 |-- country: string (nullable = true)



In [61]:
# Create parquet file
i94res.write.mode('overwrite').parquet('./data/i94res.parquet')

In [62]:
'''/* I94VISA - Visa codes collapsed into three categories:
   1 = Business
   2 = Pleasure
   3 = Student
*/'''
i94visa_data = [[1, 'Business'], [2, 'Pleasure'], [3, 'Student']]

In [63]:
# Convert to spark dataframe
i94visa=spark.createDataFrame(i94visa_data)

In [64]:
i94visa.printSchema()

root
 |-- _1: long (nullable = true)
 |-- _2: string (nullable = true)



In [65]:
# CREATE PARQUET FILE
i94visa.write.mode('overwrite').parquet('./data/i94visa.parquet')

In [66]:
us.show(3)

+---------------+----------+----------+--------+-------+-------+------------+------------------+----------+---------+---------+----------+---------+
|           City|State_Code|Median_age|Male_Pop|Fem_Pop|Ttl_Pop|Foreign-born|Avg_Household_Size|Native_Pop|Asian_Pop|Black_Pop|Latino_Pop|White_Pop|
+---------------+----------+----------+--------+-------+-------+------------+------------------+----------+---------+---------+----------+---------+
|Highlands Ranch|        CO|      39.6|   49186|  53281| 102467|        8827|              2.72|      1480|     5650|     1779|      8393|    94499|
|           Kent|        WA|      33.4|   61825|  65137| 126962|       38175|              3.06|      3651|    26168|    20450|     21928|    67918|
|        Madison|        WI|      30.7|  122596| 126360| 248956|       30090|              2.23|      2296|    23937|    20424|     19697|   204302|
+---------------+----------+----------+--------+-------+-------+------------+------------------+----------

In [67]:
i94_spark.show(3)

+------+-------+-------+-------+-------+------+-------+-----+------+-----------+
|i94res|i94port|arrdate|i94mode|depdate|i94bir|i94visa|count|gender|     admnum|
+------+-------+-------+-------+-------+------+-------+-----+------+-----------+
|   582|    XXX|  20557|   null|  20558|    34|      2|    1|  null|91904214530|
|   209|    AGA|  20552|      1|   null|  null|      2|    1|     M|47842155333|
|   209|    ATL|  20571|      1|   null|  null|      2|    1|     M|44537883633|
+------+-------+-------+-------+-------+------+-------+-----+------+-----------+
only showing top 3 rows



In [68]:
# Add i94port city and state columns to i94 dataframe
i94_spark.join(i94port, i94_spark.i94port==i94port.id, how='left').count()

3096302

In [69]:
i94_spark=i94_spark.join(i94port, i94_spark.i94port==i94port.port, how='left')

In [70]:
# Drop `id` column
i94_spark=i94_spark.drop("id")

In [71]:
i94_spark.show(3)

+------+-------+-------+-------+-------+------+-------+-----+------+-----------+----+---------+----------+
|i94res|i94port|arrdate|i94mode|depdate|i94bir|i94visa|count|gender|     admnum|port|port_city|port_state|
+------+-------+-------+-------+-------+------+-------+-----+------+-----------+----+---------+----------+
|   110|    BGM|  20550|      1|  20556|    36|      1|    1|     F|92847893530| BGM|   BANGOR|        ME|
|   108|    BGM|  20569|      1|  20571|    43|      1|    1|     M|59234302533| BGM|   BANGOR|        ME|
|   129|    BGM|  20559|      1|  20584|    67|      1|    1|     M|93553405030| BGM|   BANGOR|        ME|
+------+-------+-------+-------+-------+------+-------+-----+------+-----------+----+---------+----------+
only showing top 3 rows



In [72]:
us.show(3)

+---------------+----------+----------+--------+-------+-------+------------+------------------+----------+---------+---------+----------+---------+
|           City|State_Code|Median_age|Male_Pop|Fem_Pop|Ttl_Pop|Foreign-born|Avg_Household_Size|Native_Pop|Asian_Pop|Black_Pop|Latino_Pop|White_Pop|
+---------------+----------+----------+--------+-------+-------+------------+------------------+----------+---------+---------+----------+---------+
|Highlands Ranch|        CO|      39.6|   49186|  53281| 102467|        8827|              2.72|      1480|     5650|     1779|      8393|    94499|
|           Kent|        WA|      33.4|   61825|  65137| 126962|       38175|              3.06|      3651|    26168|    20450|     21928|    67918|
|        Madison|        WI|      30.7|  122596| 126360| 248956|       30090|              2.23|      2296|    23937|    20424|     19697|   204302|
+---------------+----------+----------+--------+-------+-------+------------+------------------+----------

In [73]:
# Join US with i94_spark to get fact table `i94non_immigrant_port_entry`
# NOTE: We use left join againt city records which may cause null values because
# we may not currently have demographic stats on all U.S. ports of entry
i94non_immigrant_port_entry=i94_spark.join(us, (upper(i94_spark.port_city)==upper(us.City)) & \
                                           (upper(i94_spark.port_state)==upper(us.State_Code)), how='left')

In [74]:
i94non_immigrant_port_entry.count()

3096302

In [75]:

i94non_immigrant_port_entry.printSchema()

root
 |-- i94res: integer (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: integer (nullable = true)
 |-- i94mode: integer (nullable = true)
 |-- depdate: integer (nullable = true)
 |-- i94bir: integer (nullable = true)
 |-- i94visa: integer (nullable = true)
 |-- count: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- admnum: long (nullable = true)
 |-- port: string (nullable = true)
 |-- port_city: string (nullable = true)
 |-- port_state: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State_Code: string (nullable = true)
 |-- Median_age: string (nullable = true)
 |-- Male_Pop: string (nullable = true)
 |-- Fem_Pop: string (nullable = true)
 |-- Ttl_Pop: string (nullable = true)
 |-- Foreign-born: string (nullable = true)
 |-- Avg_Household_Size: string (nullable = true)
 |-- Native_Pop: string (nullable = true)
 |-- Asian_Pop: string (nullable = true)
 |-- Black_Pop: string (nullable = true)
 |-- Latino_Pop: string (nullab

In [76]:
# Drop City and State_Code
i94non_immigrant_port_entry=i94non_immigrant_port_entry.drop("City","State_Code")

In [77]:
i94non_immigrant_port_entry.show(5)

+------+-------+-------+-------+-------+------+-------+-----+------+-----------+----+---------+----------+----------+--------+-------+-------+------------+------------------+----------+---------+---------+----------+---------+
|i94res|i94port|arrdate|i94mode|depdate|i94bir|i94visa|count|gender|     admnum|port|port_city|port_state|Median_age|Male_Pop|Fem_Pop|Ttl_Pop|Foreign-born|Avg_Household_Size|Native_Pop|Asian_Pop|Black_Pop|Latino_Pop|White_Pop|
+------+-------+-------+-------+-------+------+-------+-----+------+-----------+----+---------+----------+----------+--------+-------+-------+------------+------------------+----------+---------+---------+----------+---------+
|   103|    NEC|  20556|      3|  20557|    51|      2|    1|     F|  788711085| NEC|    NECHE|        ND|      null|    null|   null|   null|        null|              null|      null|     null|     null|      null|     null|
|   112|    NEC|  20573|      3|  20575|    32|      2|    1|     F|59477349333| NEC|    NEC

In [78]:
# Convert SAS arrival date to datetime format
get_date = udf(lambda x: (dt.datetime(1960, 1, 1).date() + dt.timedelta(x)).isoformat() if x else None)
i94non_immigrant_port_entry = i94non_immigrant_port_entry.withColumn("arrival_date", get_date(i94non_immigrant_port_entry.arrdate))

In [79]:
i94non_immigrant_port_entry.show(3)

+------+-------+-------+-------+-------+------+-------+-----+------+-----------+----+---------+----------+----------+--------+-------+-------+------------+------------------+----------+---------+---------+----------+---------+------------+
|i94res|i94port|arrdate|i94mode|depdate|i94bir|i94visa|count|gender|     admnum|port|port_city|port_state|Median_age|Male_Pop|Fem_Pop|Ttl_Pop|Foreign-born|Avg_Household_Size|Native_Pop|Asian_Pop|Black_Pop|Latino_Pop|White_Pop|arrival_date|
+------+-------+-------+-------+-------+------+-------+-----+------+-----------+----+---------+----------+----------+--------+-------+-------+------------+------------------+----------+---------+---------+----------+---------+------------+
|   103|    NEC|  20556|      3|  20557|    51|      2|    1|     F|  788711085| NEC|    NECHE|        ND|      null|    null|   null|   null|        null|              null|      null|     null|     null|      null|     null|  2016-04-12|
|   112|    NEC|  20573|      3|  20575|

In [80]:
#The Data could be enriched with some more info on date values
i94date=i94non_immigrant_port_entry.select(col('arrdate').alias('arrival_sasdate'),
                                   col('arrival_date').alias('arrival_iso_date'),
                                   date_format('arrival_date','M').alias('arrival_month'),
                                   date_format('arrival_date','E').alias('arrival_dayofweek'), 
                                   date_format('arrival_date', 'y').alias('arrival_year'), 
                                   date_format('arrival_date', 'd').alias('arrival_day'),
                                  date_format('arrival_date','w').alias('arrival_weekofyear')).dropDuplicates()

In [81]:
# Drop arrival_date column from the i94non_immigrant_port_entry dataframe and finally save it to parquet file 
i94non_immigrant_port_entry.drop('arrival_date').write.mode("overwrite").parquet('./data/i94non_immigrant_port_entry.parquet')

In [82]:
i94non_immigrant_port_entry.printSchema()

root
 |-- i94res: integer (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: integer (nullable = true)
 |-- i94mode: integer (nullable = true)
 |-- depdate: integer (nullable = true)
 |-- i94bir: integer (nullable = true)
 |-- i94visa: integer (nullable = true)
 |-- count: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- admnum: long (nullable = true)
 |-- port: string (nullable = true)
 |-- port_city: string (nullable = true)
 |-- port_state: string (nullable = true)
 |-- Median_age: string (nullable = true)
 |-- Male_Pop: string (nullable = true)
 |-- Fem_Pop: string (nullable = true)
 |-- Ttl_Pop: string (nullable = true)
 |-- Foreign-born: string (nullable = true)
 |-- Avg_Household_Size: string (nullable = true)
 |-- Native_Pop: string (nullable = true)
 |-- Asian_Pop: string (nullable = true)
 |-- Black_Pop: string (nullable = true)
 |-- Latino_Pop: string (nullable = true)
 |-- White_Pop: string (nullable = true)
 |-- arrival_date: string 

In [83]:
i94date.count()

30

In [84]:
i94date.show(5)

+---------------+----------------+-------------+-----------------+------------+-----------+------------------+
|arrival_sasdate|arrival_iso_date|arrival_month|arrival_dayofweek|arrival_year|arrival_day|arrival_weekofyear|
+---------------+----------------+-------------+-----------------+------------+-----------+------------------+
|          20562|      2016-04-18|            4|              Mon|        2016|         18|                17|
|          20554|      2016-04-10|            4|              Sun|        2016|         10|                16|
|          20556|      2016-04-12|            4|              Tue|        2016|         12|                16|
|          20548|      2016-04-04|            4|              Mon|        2016|          4|                15|
|          20553|      2016-04-09|            4|              Sat|        2016|          9|                15|
+---------------+----------------+-------------+-----------------+------------+-----------+------------------+
o

In [85]:
# Create temporary sql table
i94date.createOrReplaceTempView("i94date_table")

In [86]:
# Add seasons to i94 date dimension table
i94date_season=spark.sql('''select arrival_sasdate,
                         arrival_iso_date,
                         arrival_month,
                         arrival_dayofweek,
                         arrival_year,
                         arrival_day,
                         arrival_weekofyear,
                         CASE WHEN arrival_month IN (12, 1, 2) THEN 'winter' 
                                WHEN arrival_month IN (3, 4, 5) THEN 'spring' 
                                WHEN arrival_month IN (6, 7, 8) THEN 'summer' 
                                ELSE 'autumn' 
                         END AS date_season from i94date_table''')

In [87]:
i94date_season.show(3)

+---------------+----------------+-------------+-----------------+------------+-----------+------------------+-----------+
|arrival_sasdate|arrival_iso_date|arrival_month|arrival_dayofweek|arrival_year|arrival_day|arrival_weekofyear|date_season|
+---------------+----------------+-------------+-----------------+------------+-----------+------------------+-----------+
|          20562|      2016-04-18|            4|              Mon|        2016|         18|                17|     spring|
|          20554|      2016-04-10|            4|              Sun|        2016|         10|                16|     spring|
|          20556|      2016-04-12|            4|              Tue|        2016|         12|                16|     spring|
+---------------+----------------+-------------+-----------------+------------+-----------+------------------+-----------+
only showing top 3 rows



In [88]:
i94date_season.printSchema()

root
 |-- arrival_sasdate: integer (nullable = true)
 |-- arrival_iso_date: string (nullable = true)
 |-- arrival_month: string (nullable = true)
 |-- arrival_dayofweek: string (nullable = true)
 |-- arrival_year: string (nullable = true)
 |-- arrival_day: string (nullable = true)
 |-- arrival_weekofyear: string (nullable = true)
 |-- date_season: string (nullable = false)



In [89]:
# Save i94date dimension to parquet file partitioned by year and month:
i94date_season.write.mode("overwrite").partitionBy("arrival_year", "arrival_month").parquet('./data/i94date.parquet')

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

The parquet files we wrote could be used for any columnar database. As the data is not that big it could be even simply just storaged as a csv in AWS S3. The Star Schema model was chosen because it will be easier for Data Analysts and Data Scientists to understand and apply queries with best performance outcomes and flexibility without constructing complex joins over several tables.

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

- Create the dimensions (i94port, i94visa, i94res, i94mode) from i94_SAS_Labels_Descriptions.SAS file. *NOTE: Once they're created it does not have to be included in future Data Pipeline schedules because these are essentially master records which do not frequently get added or changed on the dimension tables.
- Read US Cities Demo dataset file to form us_spark dataframe
- Create 'us_race_cnt' from us_spark
- Drop columns we don't need and drop duplicate rows from us_spark
- Join us_spark with us_race_cnt to form US data set
- Change state code column name to state_code and other similar problems to avoid parquet complications and drop the state column
- Write (and overwrite) transformed US dataset onto parquet file
- Read i94 non-immigration dataset to form i94_spark dataframe
- Convert numbers to longtype and integertype
- Drop duplicate rows
- Read i94port dimension parquet file so we can use it to join with i94_spark. This will add i94port city and state columns to i94_spark dataframe
- Drop id column from i94_spark dataframe
- Join US with i94_spark to get fact table i94non_immigrant_port_entry
- Add iso date format column arrival_date inside the i94non_immigrant_port_entry dataframe by using custom function.
- Create time dimension from i94non_immigrant_port_entry and save to parquet file.
- Drop arrival_date column from i94non_immigrant_port_entry and save it to parquet file.
- Add seasons to i94date_seasons dataframe.
- Save i94date_seasons to parquet file partitioned by year and month.

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

The data pipeline is built inside the ETL.py file included with this Capstone Project.

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [93]:
# Perform quality checks here

if us_spark.count() > 0:
    print('Passed reading data file.')
else:
    print('Seems to be nothing in file!')

if us.count() == us_race_count.count():
    print('Transformation went perfect.')
else:
    print('Inconsistant data between both dataframes!')

if i94_spark.count() > 0:
    print('Passed reading data file.')
else:
    print('Seems to be nothing in file!')
    
if i94_spark.count() == i94non_immigrant_port_entry.count():
    print('Transformation went perfect.')
else:
    print('Inconsistant data between both dataframes!')

Passed reading data file.
Transformation went perfect.
Passed reading data file.
Transformation went perfect.


#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.


'''

i94date
arrival_sasdate int PK

> Non-immigrant arrival date in the USA (SAS date numeric field) from `arrdate` i94 non-immigration data
arrival_isodate date

> converted to iso date format (YYYY-MM-DD) from arrival_sasdate
arrival_month int arrival_day int arrival_year int arrival_week int

> week of year
arrival_season string

> winter, spring, summer or autumn seasons

i94port
id string PK

> i94 port 3 character code of destination USA city
city string state string

i94visa
id int PK

> i94 visa code number stating reason for immigration from i94 non-immigration data
reason string

i94res
id int PK

> i94 3 digit code of nationality from i94 non-immigration data
country string

i94mode
id int PK

> i94 1 digit mode (plane, boat, etc) of travel code from i94 non-immigration data
transport string

i94non_immigrant_port_entry
admnum int PK

> Admission Number from i94 non-immigration data
arrdate int FK >- i94date.arrival_sasdate

> arrival date in the USA (SAS date numeric field) from i94 non-immigration data
depdate int

> Departure Date from the USA. It is a SAS date numeric field from i94 non-immigration data
port_id string FK >- i94port.id

> 3 character code of destination USA city from i94 non-immigration data
visa_id int FK >- i94visa.id

> reason for immigration from i94 non-immigration data
res_id int FK >- i94res.id

> 3 digit code of nationality from i94 non-immigration data
mode_id int FK >- i94mode.id

> 1 digit mode (plane, boat, etc) of travel code from i94 non-immigration data
age int

> Age of non-immigrant in years from i94 non-immigration data
gender string

> Non-immigrant sex from i94 non-immigration data
cnt_of_one int

> count of one per row used for statistical metrics from i94 non-immigration data
median_age float

> Median age of population in city and state from US cities demographics data
male_pop int

> Male population of city and state from US cities demographics data
female_pop int

> Female population of city and state from US cities demographics data
ttl_pop int

> Total population of city and state from US cities demographics data
foreign_born_pop int

> Foreign born population of city and state from US cities demographics data
avg_household_size float

> Average household size of city and state from US cities demographics data
american_indian_alaskan_native_pop int

> Aerican Indian population of city and state from US cities demographics data
asian_pop int

> Asian population of city and state from US cities demographics data
black_african_american_pop int

> Black population of city and state from US cities demographics data
hispanic_pop int

> Hispanic population of city and state from US cities demographics data
white_pop int

> White population of city and state from US cities demographics data

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
> Because we are dealing with big data and cloud technologies for solutions it made economical sense to use opensource Apache Spark and Python tools that can be easily ported over to cloud solution such as AWS or others. Pandas is always a good way for quick checks but is not that usefull when the data becomes bigger. Writing the data as parquet files improves efficiencies if we are going to write out the data to columnar oriented tables in any datalake as redshift etc. eg. It is even more efficient in using it with AWS EMR because of the underlying hadoop ecosystem.
* Propose how often the data should be updated and why.
> Dimenstion tables only have to be updated when a new category is created by I94. However, the I94 non-immigrant port of entry data along with the time dimension table (i94date) can be updated every month. The US Cities Demographics data is updated every ten years according to https://www.usa.gov/statistics. So, the new US Cities Demographics data set maybe coming after year 2020. And may need updating after one year or two years as of 2019.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 > Deploy this Spark solution on a cluster using AWS (EMR cluster) and use S3 for data and parquet file storage or alternatively a data lake like Amazon Redshift.. AWS will easily scale when data increases by 100x. The scaling in a cloud will not be the bootleneck, it is more about the question to adapt the table schema for the purpose need. 
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 > Use Apache Airflow to schedule queries. This will have also the nice seideeffect that airflow can reshedule if something does not work out or even give you an information about that somethign did not work out. Using aurflow for operational tasking gives you several abilities to get nbetter insghts (eg. how long did my ETL took?)
 * The database needed to be accessed by 100+ people.
 > The saved parquet files can be bulk copied over to AWS Redshift cluster where it can scale big data requirements and has 'massively parallel' and 'limitless concurrency' for thousands of concurrent queries executed by users.

In [ ]:
# Answer a question with the new dataset

In [94]:
# Do all imports and installs here
import pandas as pd
import os
from pyspark.sql import SparkSession
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = "/opt/conda/bin:/opt/spark-2.4.3-bin-hadoop2.7/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/lib/jvm/java-8-openjdk-amd64/bin"
os.environ["SPARK_HOME"] = "/opt/spark-2.4.3-bin-hadoop2.7"
os.environ["HADOOP_HOME"] = "/opt/spark-2.4.3-bin-hadoop2.7"
spark = SparkSession.builder.getOrCreate()
i94non_immigrant_port_entry = spark.read.load('./data/i94non_immigrant_port_entry.parquet')
print("complete")

complete


In [95]:
df_i94non_immigrant_port_entry_sample = i94non_immigrant_port_entry.sample(0.35)
df_i94non_immigrant_port_entry_sample.count()

1083121

In [96]:
dfp_i94non_immigrant_port_entry = df_i94non_immigrant_port_entry_sample.toPandas()

In [97]:
dfp_i94non_immigrant_port_entry.columns

Index(['i94res', 'i94port', 'arrdate', 'i94mode', 'depdate', 'i94bir',
       'i94visa', 'count', 'gender', 'admnum', 'port', 'port_city',
       'port_state', 'Median_age', 'Male_Pop', 'Fem_Pop', 'Ttl_Pop',
       'Foreign-born', 'Avg_Household_Size', 'Native_Pop', 'Asian_Pop',
       'Black_Pop', 'Latino_Pop', 'White_Pop'],
      dtype='object')

In [98]:
dfp_i94non_immigrant_port_entry[["Latino_Pop"]] = dfp_i94non_immigrant_port_entry[["Latino_Pop"]].apply(pd.to_numeric)

In [99]:
dfp_i94non_immigrant_port_entry.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1083121 entries, 0 to 1083120
Data columns (total 24 columns):
i94res                1083121 non-null int32
i94port               1083121 non-null object
arrdate               1083121 non-null int32
i94mode               1083028 non-null float64
depdate               1033127 non-null float64
i94bir                1082876 non-null float64
i94visa               1083121 non-null int32
count                 1083121 non-null int32
gender                937810 non-null object
admnum                1083121 non-null int64
port                  1083121 non-null object
port_city             1083121 non-null object
port_state            1083121 non-null object
Median_age            835306 non-null object
Male_Pop              835306 non-null object
Fem_Pop               835306 non-null object
Ttl_Pop               835306 non-null object
Foreign-born          832152 non-null object
Avg_Household_Size    832152 non-null object
Native_Pop            

In [100]:
# On which Port do we have the most Latino Population
dfp_i94non_immigrant_port_entry[dfp_i94non_immigrant_port_entry["Latino_Pop"] > 10000]
df = dfp_i94non_immigrant_port_entry.groupby(['i94port']).sum()
df.sort_values(by=['Latino_Pop'], ascending=False).head(5)

,i94res,arrdate,i94mode,depdate,i94bir,i94visa,count,admnum,Latino_Pop
i94port,,,,,,,,,
NYC,44049837.0,3.498593e+09,171254.0,3.371438e+09,6960366.0,320253.0,170165.0,11974371751808188,4.228813e+11
LOS,33729767.0,2.232039e+09,109490.0,2.153283e+09,4504396.0,201741.0,108563.0,7719655549416933,2.102574e+11
MIA,55217560.0,2.478909e+09,122836.0,2.389539e+09,5028821.0,230482.0,120571.0,9440925255493812,3.857573e+10
HOU,14080021.0,7.316890e+08,36374.0,6.953866e+08,1563491.0,62299.0,35588.0,2759338405495830,3.658973e+10
CHI,11308624.0,9.349573e+08,46021.0,8.844225e+08,1977324.0,77353.0,45475.0,3286648526740453,3.573084e+10


In [101]:
#On which date did we had the most Visa applies
df = dfp_i94non_immigrant_port_entry.groupby(['arrdate']).sum()
df.sort_values(by=['i94visa'], ascending=False).head(5)

,i94res,i94mode,depdate,i94bir,i94visa,count,admnum,Latino_Pop
arrdate,,,,,,,,
20573,13561599,47891.0,894871336.0,1857766.0,86559,45084,3286022937723095,2.835053e+10
20574,13068936,47832.0,871190810.0,1836935.0,82715,44297,3223239876381553,3.147453e+10
20572,12875539,44225.0,826483192.0,1792379.0,81045,41991,3093580152067753,3.329892e+10
20559,12498355,46967.0,787343394.0,1686515.0,75483,40220,2855783411044347,2.816699e+10
20566,12119110,42183.0,756558393.0,1613118.0,74403,38565,2802619680951964,3.048849e+10
